In [1]:
import tensorflow as tf
import numpy as np
import utils

__author__="Olivares Castillo José Luis"

print("TensorFlow",tf.VERSION)
if tf.test.gpu_device_name():
    print("GPU disponible")

tf.logging.set_verbosity(tf.logging.INFO)

/home/olivares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow 1.7.0


In [2]:
src_lex, trg_lex = utils.get_lexicon("en-it.test")

In [3]:
len(set(src_lex)), len(set(trg_lex))

(1500, 1849)

In [4]:
src_matrix = utils.open_file("en-crawl")
words_src, source_vec = utils.read(src_matrix)

In [5]:
len(words_src)

2000000

In [10]:
tm = list(set(src_lex))
s=0
for i in tm:
    if i not in words_src:
        print(i)
        s+=1
print(s)

ecevit
jaruzelski
tamayo
ekaterinburg
ogoni
hélène
einaudi
bavarians
méndez
fyrom
allawi
sète
inés
gbagbo
raynal
jaurès
véronique
heysel
valenciennes
kouchner
bogotá
michelis
antonello
daghestan
rosati
unclos
mossadeq
clarín
walesa
isfahan
vukovar
igad
antioquia
ngawang
izetbegovic
tegucigalpa
chandrika
unrwa
tindouf
cpmp
bouteflika
togolese
kumaratunga
rafah
fassa
radovan
lindh
rovaniemi
romanies
andreotti
ravalomanana
nazarbayev
brueghel
guterres
ingushetia
nlg
tornio
schreyer
jiabao
lyudmila
prizren
kfor
ratko
katyn
eeurope
wilfredo
gaborone
papadopoulos
lahoud
kostunica
pilger
vélez
kursk
jörg
ncbs
sudeten
malpensa
kabylie
suharto
unodc
megawati
agusta
saakashvili
scowcroft
michèle
sorbonne
bassam
atocha
beckenbauer
thüringen
elmar
idriss
beazley
vercelli
bessarabia
hristo
rehn
karamanlis
caretta
lumumba
matera
genk
ceausescu
macra
interreg
gaullists
yalta
euskadi
billingham
nanterre
mendieta
núñez
csps
113


In [13]:
"tegucigalpa" in words_src

False